# The Movie DataBase (TMDB)

In [32]:
import pandas as pd
import numpy as np
import json

In [12]:
movies= pd.read_csv('tmdb_5000_movies.csv')
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [10]:
credits= pd.read_csv('tmdb_5000_credits.csv')
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## Normalizzazione

#### Creo tabella genere

In [164]:
genere=pd.DataFrame()
for film in list(range(0,len(movies['genres']))):
    tabjson=pd.read_json(movies['genres'][film])
    genere=pd.concat([genere, tabjson])
genere = genere.set_index(['id'])
genere.head()

,name
id,
28,Action
12,Adventure
14,Fantasy
878,Science Fiction
12,Adventure


#### Creo tabella genere_id

In [166]:
genere_id=pd.DataFrame()
for film in list(range(0,len(movies['genres']))):
    tabjson=pd.read_json(movies['genres'][film])
    idfilm=movies['id'][film]
    tabjson['movie_id']=idfilm
    genere_id=pd.concat([genere_id, tabjson])
del genere_id['name']
genere_id = genere_id.set_index(['movie_id', 'id'])
genere_id.head()

Empty DataFrame
Columns: []
Index: [(19995, 28.0), (19995, 12.0), (19995, 14.0), (19995, 878.0), (285, 12.0)]

#### Creo tabella keywords

In [162]:
keywords=pd.DataFrame()
for film in list(range(0,len(movies['keywords']))):
    tabjson=pd.read_json(movies['keywords'][film])
    keywords=pd.concat([keywords, tabjson])
keywords = keywords.set_index(['id'])
keywords.head()

,name
id,
1463,culture clash
2964,future
3386,space war
3388,space colony
3679,society


#### Creo tabella keywords_id

In [167]:
keywords_id=pd.DataFrame()
for film in list(range(0,len(movies['keywords']))):
    tabjson=pd.read_json(movies['keywords'][film])
    idfilm=movies['id'][film]
    tabjson['movie_id']=idfilm
    keywords_id=pd.concat([keywords_id, tabjson])
del keywords_id['name']
keywords_id = keywords_id.set_index(['movie_id', 'id'])
keywords_id.head()

Empty DataFrame
Columns: []
Index: [(19995, 1463.0), (19995, 2964.0), (19995, 3386.0), (19995, 3388.0), (19995, 3679.0)]

#### Creo tabella prod_companies

In [168]:
prod_companies=pd.DataFrame()
for film in list(range(0,len(movies['production_companies']))):
    tabjson=pd.read_json(movies['production_companies'][film])
    prod_companies=pd.concat([prod_companies, tabjson])
prod_companies = prod_companies.set_index(['id'])
prod_companies.head()

,name
id,
289,Ingenious Film Partners
306,Twentieth Century Fox Film Corporation
444,Dune Entertainment
574,Lightstorm Entertainment
2,Walt Disney Pictures


#### Creo tabella prod_companies_id

In [170]:
prod_companies_id=pd.DataFrame()
for film in list(range(0,len(movies['production_companies']))):
    tabjson=pd.read_json(movies['production_companies'][film])
    idfilm=movies['id'][film]
    tabjson['movie_id']=idfilm
    prod_companies_id=pd.concat([prod_companies_id, tabjson])
del prod_companies_id['name']
prod_companies_id = prod_companies_id.set_index(['movie_id', 'id'])
prod_companies_id.head()

Empty DataFrame
Columns: []
Index: [(19995, 289.0), (19995, 306.0), (19995, 444.0), (19995, 574.0), (285, 2.0)]

#### Creo tabella prod_countries

In [171]:
prod_countries=pd.DataFrame()
for film in list(range(0,len(movies['production_countries']))):
    tabjson=pd.read_json(movies['production_countries'][film])
    prod_countries=pd.concat([prod_countries, tabjson])
prod_countries = prod_countries.set_index(['iso_3166_1'])
prod_countries.head()

,name
iso_3166_1,
US,United States of America
GB,United Kingdom
US,United States of America
GB,United Kingdom
US,United States of America


#### Creo tabella prod_countries_id

In [ ]:
prod_countries=pd.DataFrame()
for film in list(range(0,len(movies['production_countries']))):
    tabjson=pd.read_json(movies['production_countries'][film])
    idfilm=movies['id'][film]
    tabjson['movie_id']=idfilm
    prod_countries=pd.concat([prod_countries, tabjson])
prod_countries = prod_countries.set_index(['movie_id', 'iso_3166_1'])
prod_countries.head()

#### Creo tabella spoken_languages

In [153]:
spoken_languages=pd.DataFrame()
for film in list(range(0,len(movies['spoken_languages']))):
    tabjson=pd.read_json(movies['spoken_languages'][film])
    idfilm=movies['id'][film]
    tabjson['movie_id']=idfilm
    spoken_languages=pd.concat([spoken_languages, tabjson])
spoken_languages = spoken_languages.set_index(['movie_id', 'iso_636_1'])
spoken_languages.head()

KeyError: 'iso_636_1'

1- For each movie, compute the number of cast members

2- How many movies do not have a homepage?

3- For each year, how many movies do not have a homepage?

4- Extract the domain of each homepage.

5- Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary).

6- For each movie, compute the gross margin (difference between revenue and budget)

7- For each movie, compute the number of crew members

8- For each movie, compute the number of directors

9- For each language, compute the number of movies where such language is spoken.

10- For each company and each decade, compute the overall revenue

11- For each decade, compute the company with maximum revenue

12- In each year, how many movies have revenue smaller than the budget?



1- Distribute the revenue according to the order of appearance in a movie. Assume that the i-th actor contributes twice as much as the (i+1)-th actor to the revenue.

2- For each actor find the total revenue attributed to him/her.

3- Find the actor that is responsible for the most overall revenue.



1- For each movie, compute the ratio between males and females in the cast

2- For each movie, compute the ratio between the attributed revenue of males and females in the cast

3- Find the director that has the highest average ratio computed in the previous point.
